# Tip Inventory Consolidation

In [1]:
import numpy as np
import random

## Workcell Setup

In [2]:
# === Configuration ===
script_mode = "simulation"  # or "execution"
liquid_handler_choice = "star"  # star | ot2 | evo100 | etc.

In [3]:
# For development: auto-reload modules
%load_ext autoreload
%autoreload 2

import logging
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.visualizer.visualizer import Visualizer

# === Liquid handler configuration ===
liquid_handler_config = {
    "star": {
        "deck": ("pylabrobot.resources.hamilton", "STARLetDeck"),
        "execution": ("pylabrobot.liquid_handling.backends", "STARBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "vantage": {
        "deck": ("pylabrobot.resources.hamilton", "VantageDeck"),
        "execution": ("pylabrobot.liquid_handling.backends", "VantageBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "ot2": {
        "deck": ("pylabrobot.resources.opentrons", "OTDeck"),
        "execution": {
            "module": "pylabrobot.liquid_handling.backends",
            "class": "OpentronsBackend"
        },
        "simulation": {
            "module": "pylabrobot.liquid_handling.backends",
            "class": "LiquidHandlerChatterboxBackend",
            "args": { "num_channels": 1 }
        }
    },
    "evo100": {
        "deck": ("pylabrobot.resources.tecan", "EVO100Deck"),
        "execution": ("pylabrobot.liquid_handling.backends", "EVOBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "evo150": {
        "deck": ("pylabrobot.resources.tecan", "EVO150Deck"),
        "execution": ("pylabrobot.liquid_handling.backends", "EVOBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "evo200": {
        "deck": ("pylabrobot.resources.tecan", "EVO200Deck"),
        "execution": ("pylabrobot.liquid_handling.backends", "EVOBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
}

# === Deck loading ===
lh_entry = liquid_handler_config.get(liquid_handler_choice)
if lh_entry is None:
    raise ValueError(f"Unknown liquid handler: {liquid_handler_choice}")

deck_module, deck_class = lh_entry["deck"]
exec(f"from {deck_module} import {deck_class} as Deck")
deck = Deck()

# === Backend loading ===
backend_entry = lh_entry.get(script_mode)
if backend_entry is None:
    raise ValueError(f"No backend configured for mode '{script_mode}' in '{liquid_handler_choice}'")

if isinstance(backend_entry, tuple):
    backend_module, backend_class = backend_entry
    backend_args = {}
elif isinstance(backend_entry, dict):
    backend_module = backend_entry["module"]
    backend_class = backend_entry["class"]
    backend_args = backend_entry.get("args", {})
else:
    raise ValueError(f"Invalid backend entry format: {backend_entry}")

exec(f"from {backend_module} import {backend_class} as Backend")
backend = Backend(**backend_args)

# === Create LiquidHandler ===
lh = LiquidHandler(backend=backend, deck=deck)

# === Logging setup ===
logger = logging.getLogger("pylabrobot")
logger.setLevel(logging.DEBUG)

In [4]:
lh = LiquidHandler(backend=backend, deck=deck)

await lh.setup()
vis = Visualizer(resource=lh)
await vis.setup()
if script_mode == 'simulation':
    from pylabrobot.resources import set_tip_tracking, set_volume_tracking
    set_tip_tracking(True), set_volume_tracking(False);
else:
    await lh.backend.disable_cover_control()
    await lh.backend.move_all_channels_in_z_safety()
    lh.backend.allow_firmware_planning = True # very powerful
    lh.backend.read_timeout = 240 # give your commands more time

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource teaching_carrier was assigned to the liquid handler.
Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


## Deck Setup

In [5]:
if liquid_handler_choice == "star" :

    from pylabrobot.resources import (
        TIP_CAR_480_A00, HTF, STF, TIP_50ul
    )
    
    tip_carrier = TIP_CAR_480_A00(name="tip carrier")
    
    tip_carrier[2] = tip_rack_1000ul_3 = HTF(name="tip_rack_1000ul_3", with_tips=False)
    tip_carrier[1] = tip_rack_1000ul_2 = HTF(name="tip_rack_1000ul_2")
    tip_carrier[0] = tip_rack_1000ul_1 = HTF(name="tip_rack_1000ul_1")
    
    lh.deck.assign_child_resource(tip_carrier, rails=15)
    
    tip_carrier_2 = TIP_CAR_480_A00(name="tip carrier 2")
    
    tip_carrier_2[2] = tip_rack_50ul_3 = TIP_50ul(name="tip_rack_50ul_3", with_tips=False)
    tip_carrier_2[1] = tip_rack_50ul_2 = TIP_50ul(name="tip_rack_50ul_2")
    tip_carrier_2[0] = tip_rack_50ul_1 = TIP_50ul(name="tip_rack_50ul_1")
    
    lh.deck.assign_child_resource(tip_carrier_2, rails=22)

    n = 55
    numbers = random.sample(range(96), k=n)
    
    _ = [tip_rack_50ul_1.children[idx].tracker.remove_tip() for idx in numbers]
    _ = [tip_rack_50ul_1.children[idx].tracker.commit() for idx in numbers]

Resource tip carrier was assigned to the liquid handler.
Resource tip carrier was assigned to the liquid handler.
Resource tip carrier 2 was assigned to the liquid handler.
Resource tip carrier 2 was assigned to the liquid handler.


In [6]:
if liquid_handler_choice == "ot2" :

    from pylabrobot.resources import (
        opentrons_96_filtertiprack_1000ul,
        opentrons_96_filtertiprack_20ul
    )

    tip_rack_1000ul_3 = opentrons_96_filtertiprack_1000ul(name="tip_rack_1000ul_3")#, with_tips=False)
    tip_rack_1000ul_2 = opentrons_96_filtertiprack_1000ul(name="tip_rack_1000ul_2")
    tip_rack_1000ul_1 = opentrons_96_filtertiprack_1000ul(name="tip_rack_1000ul_1")

    lh.deck.assign_child_at_slot(tip_rack_1000ul_3, slot=8)
    lh.deck.assign_child_at_slot(tip_rack_1000ul_2, slot=5)
    lh.deck.assign_child_at_slot(tip_rack_1000ul_1, slot=2)

    tip_rack_20ul_3 = opentrons_96_filtertiprack_20ul(name="tip_rack_20ul_3")#, with_tips=False)
    tip_rack_20ul_2 = opentrons_96_filtertiprack_20ul(name="tip_rack_20ul_2")
    tip_rack_20ul_1 = opentrons_96_filtertiprack_20ul(name="tip_rack_20ul_1")

    lh.deck.assign_child_at_slot(tip_rack_20ul_3, slot=9)
    lh.deck.assign_child_at_slot(tip_rack_20ul_2, slot=6)
    lh.deck.assign_child_at_slot(tip_rack_20ul_1, slot=3)

    n = 55
    numbers = random.sample(range(96), k=n)
    
    _ = [tip_rack_20ul_1.children[idx].tracker.remove_tip() for idx in numbers]
    _ = [tip_rack_20ul_1.children[idx].tracker.commit() for idx in numbers]

In [7]:
if liquid_handler_choice == "evo150" :

    from pylabrobot.resources import (
        opentrons_96_filtertiprack_1000ul,
        opentrons_96_filtertiprack_20ul
    )

In [8]:
[
    [xd.get_identifier() for xd in x ]
     for x in tip_rack_50ul_3.traverse(
         batch_size=8,
         direction="left"
     )       
]
# help(tip_rack_50ul_3.traverse)

[['A12', 'A11', 'A10', 'A9', 'A8', 'A7', 'A6', 'A5'],
 ['A4', 'A3', 'A2', 'A1', 'B12', 'B11', 'B10', 'B9'],
 ['B8', 'B7', 'B6', 'B5', 'B4', 'B3', 'B2', 'B1'],
 ['C12', 'C11', 'C10', 'C9', 'C8', 'C7', 'C6', 'C5'],
 ['C4', 'C3', 'C2', 'C1', 'D12', 'D11', 'D10', 'D9'],
 ['D8', 'D7', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1'],
 ['E12', 'E11', 'E10', 'E9', 'E8', 'E7', 'E6', 'E5'],
 ['E4', 'E3', 'E2', 'E1', 'F12', 'F11', 'F10', 'F9'],
 ['F8', 'F7', 'F6', 'F5', 'F4', 'F3', 'F2', 'F1'],
 ['G12', 'G11', 'G10', 'G9', 'G8', 'G7', 'G6', 'G5'],
 ['G4', 'G3', 'G2', 'G1', 'H12', 'H11', 'H10', 'H9'],
 ['H8', 'H7', 'H6', 'H5', 'H4', 'H3', 'H2', 'H1']]

In [9]:
[
    [xd.get_identifier() for xd in x ]
     for x in tip_rack_50ul_3.traverse(
         batch_size=8,
         direction="down_left"
     )       
]
# help(tip_rack_50ul_3.traverse)

[['A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12'],
 ['A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11'],
 ['A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1']]

In [9]:
tip_rack_50ul_3.children[0].get_identifier()

'A1'

In [9]:
n = 40

random_source_numbers = random.sample(range(96), k=n)
dest_tip_spot_iterator_reversed = random.sample(range(96), k=n)

for idx in range(0, n, lh.backend.num_channels):
    print(idx, idx+lh.backend.num_channels)

    source_tip_spots = [
        tip_rack_1000ul_1.children[random_source_numbers[i]]
        for i in range(idx, idx+lh.backend.num_channels)
    ]

    destination_tip_spots = [
        tip_rack_1000ul_3.children[random_dest_numbers[i]]
        for i in range(idx, idx+lh.backend.num_channels)
    ]

    await lh.pick_up_tips(
        source_tip_spots,
    )

    await lh.drop_tips(
        destination_tip_spots,
        )

    
# if script_mode == "simulation":
#     _ = [tip_rack_1000ul_1.children[idx].tracker.remove_tip() for idx in numbers]
#     _ = [tip_rack_1000ul_1.children[idx].tracker.commit() for idx in numbers]

0 8
Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_1000ul_1_tipspot_6_1 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tip_rack_1000ul_1_tipspot_4_0 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tip_rack_1000ul_1_tipspot_2_5 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tip_rack_1000ul_1_tipspot_9_2 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p4: tip_rack_1000ul_1_tipspot_1_1 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p5: tip_rack_1000ul_1_tipspot_5_7 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p6: tip_rack_1000ul_1_tipspot_11_3 0,0,0            HamiltonTi

In [ ]:
n = 40
numbers = random.sample(range(96), k=n)

if script_mode == "simulation":
    _ = [tip_rack_1000ul_2.children[idx].tracker.remove_tip() for idx in numbers]
    _ = [tip_rack_1000ul_2.children[idx].tracker.commit() for idx in numbers]

In [ ]:
from typing import List, Any, Generator
from pylabrobot.resources import TipRack


async def consolidate_tip_inventory(
    lh,
    ignore_tiprack_list: List[str] = ["teaching_tip_rack"]
    ):
    """
    Consolidate partial tip racks on the deck by redistributing tips.

    This function identifies partially-filled tip racks (excluding any in
    `ignore_tiprack_list`) and consolidates their tips into as few tip racks
    as possible, grouped by tip model. Tips are moved efficiently to minimize
    pipetting steps, avoiding redundant visits to the same drop columns.

    Args:
        lh: The liquid handler instance providing access to deck resources and
            pick/drop operations.
        ignore_tiprack_list: List of tip rack names to exclude from consolidation.

    Returns:
        None. The function performs in-place tip redistribution via async pick/drop.
    """
    
    def merge_sublists(
        lists: List[List[int]],
        max_len: int
        ) -> List[List[int]]:
        """
        Merge adjacent sublists if combined length <= max_len,
          without splitting sublists."""
        merged, buffer = [], []
    
        for sublist in lists:
            if not sublist:
                continue  # skip empty sublists
    
            if len(buffer) + len(sublist) <= max_len:
                buffer.extend(sublist)
            else:
                if buffer:
                    merged.append(buffer)
                buffer = sublist  # start new buffer
    
        if buffer:
            merged.append(buffer)
    
        return merged

    def divide_list_into_chunks(
            list_l: List[Any],
            chunk_size: int
        ) -> Generator[List[Any], None, None]:
        """
        Divides a list into smaller chunks of a specified size.
    
        Parameters:
        - list_l (List[Any]): The list to be divided into chunks.
        - chunk_size (int): The size of each chunk.
    
        Returns:
        - Generator[List[Any], None, None]: A generator that yields chunks of the list.
        """
        for i in range(0, len(list_l), chunk_size):
            yield list_l[i:i + chunk_size]
    
    all_tipracks_on_deck_list = [
        item for item in lh.get_all_children()
        if isinstance(item, TipRack) and item.name not in ignore_tiprack_list
    ]
    
    clusters_by_model = {}
    
    for idx, tip_rack in enumerate(all_tipracks_on_deck_list):
    
        # Only consider partially-filled tip_racks
        tip_status = [
                tip_spot.tracker.has_tip
                for tip_spot in tip_rack.children
            ]
        partially_filled = any(tip_status) and not all(tip_status)
    
        if partially_filled:
    
            tipspots_w_tips = [i for b, i in zip(tip_status, tip_rack.children) if b]
    
            # Identify model by hashed unique physical characteristics
            current_model = hash(tipspots_w_tips[0].tracker.get_tip())
            
            num_empty_tipspots = len(tip_status) - len(tipspots_w_tips)
            
            sanity_check = all(
                hash(tip_spot.tracker.get_tip())==current_model 
                for tip_spot in tipspots_w_tips[1:]
            )
            
            if sanity_check:
                clusters_by_model.setdefault(current_model, []).append((tip_rack, num_empty_tipspots))
    
    # Sort partially-filled tipracks by minimal fill_len
    for model, rack_list in clusters_by_model.items():
        rack_list.sort(key=lambda x: x[1])
    
    # Consolidate one tip model at a time across all tip_racks of that model
    for model, rack_list in clusters_by_model.items():

        print(f"Consolidating:\n - {', ' .join([rack.name for rack, num in rack_list])}")
   
        all_tip_spots_list = [tip for tip_rack, _ in rack_list for tip in tip_rack.children]
        
        # 1: Record current tip state
        current_tip_presence_list = [tip_spot.has_tip() for tip_spot in all_tip_spots_list]
    
        # 2: Generate target/consolidated tip state
        total_length = len(all_tip_spots_list)
        num_tips_per_model = sum(current_tip_presence_list)
    
        target_tip_presence_list = [
            True if i < num_tips_per_model else False for i in range(total_length)
            i < num_tips_per_model for i in range(total_length)
        ]
    
        # 3: Calculate tip_spots involved in tip movement
        tip_movement_list = [
            c - t for c, t in zip(current_tip_presence_list, target_tip_presence_list)
        ]
    
        tip_origin_indices = [i for i, v in enumerate(tip_movement_list) if v == 1]
        all_origin_tip_spots = [all_tip_spots_list[idx] for idx in tip_origin_indices]
    
        tip_target_indices = [i for i, v in enumerate(tip_movement_list) if v == -1]
        all_target_tip_spots = [all_tip_spots_list[idx] for idx in tip_target_indices]
    
        # 4: Cluster target tip_spots by x-coordinate
        target_tip_clusters_by_x = {}
        for idx, tip_spot in enumerate(all_target_tip_spots):
            x = round(tip_spot.location.x, 3)
            target_tip_clusters_by_x.setdefault(x, []).append(tip_spot)

        # Only continue if tip_racks are not already consolidated
        if len(target_tip_clusters_by_x) > 0:

            current_tip_model = all_origin_tip_spots[0].tracker.get_tip()

            # Ensure there are channels that can pick up the tip model
            num_channels_available = len([
                c for c in range(lh.backend.num_channels)
                if lh.backend.can_pick_up_tip(c, current_tip_model)
            ])

            # 5: Optimise speed 
            if num_channels_available > 0:

                # by aggregating drop columns i.e. same drop column should not be visited twice!
                if num_channels_available >= 8: # physical constraint of tip_rack's having 8 rows
                    
                    merged_target_tip_clusters = merge_sublists(
                        target_tip_clusters_by_x.values(),
                        max_len = 8
                    )

                # by chunking drop columns into size of available channels
                else:
                    
                    merged_target_tip_clusters = list(divide_list_into_chunks(
                        all_target_tip_spots,
                        chunk_size = num_channels_available
                    ))
                    
                len_transfers = len(merged_target_tip_clusters)
                
                # 6: Execute tip movement/consolidation
                for idx, target_tip_spots in enumerate(merged_target_tip_clusters):
                    print(f"     - tip transfer cycle: {idx} / {len_transfers-1}")
                    num_channels = len(target_tip_spots)
                    use_channels = list(range(num_channels))
    
                    origin_tip_spots = [all_origin_tip_spots.pop(0) for idx in range(num_channels)]
    
                    await lh.pick_up_tips(
                        origin_tip_spots,
                        use_channels=use_channels
                    )
        
                    await lh.drop_tips(
                        target_tip_spots,
                        use_channels=use_channels
                    )
            else:
                print(f"Tips already optimally consolidated!")

        else:
            raise ValueError(f"No channel capable of handling tips on deck: {current_tip_model}")

In [ ]:
lh.backend.num_channels

In [ ]:
await consolidate_tip_inventory(
    lh=lh
)

## 3- sorting algorithm

### 3.0- identify tipracks

### 3.1- cluster tipracks by model

### 3.2- store only partially-filled tipracks

### 3.3- sort partially-filled tipracks by minimal fill_len

### 3.4- vector calculations to assess `add` & `remove` TipSpots

### 3.5- Cluster `add` by x-coordiate

### 3.6- Merge `add` clusters if len(clusters) <= len(use_channels)

In [ ]:
import random
import matplotlib.pyplot as plt

# Parameters
total_length = 182
num_tips = 106

# Generate current tip list
random.seed(42)
current_tips_list = all_tip_presence_list

# Generate target tip list
target_tips_list = target_tips_list

# Compute movement list
tip_movement_list = [
    c - t for c, t in zip(current_tips_list, target_tips_list)
]

# Stack lists for visualization
tips_matrix = [current_tips_list, target_tips_list, tip_movement_list]

# Convert to 2D list of floats for imshow
tips_matrix_float = [[float(x) for x in row] for row in tips_matrix]

# Plot using matplotlib
fig, ax = plt.subplots(figsize=(12, 3))
cax = ax.imshow(tips_matrix_float, cmap="bwr", aspect="auto", vmin=-1, vmax=1)
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(["Current", "Target", "Movement"])
ax.set_xticks([])
ax.set_title("Tip Movement Overview (No NumPy)")
plt.colorbar(cax, orientation='vertical', label='Tip Delta')
plt.tight_layout()
plt.show()
